# Прогнозирование цен потребительского ритейла по тестовой выборке на основе глубоких нейронных сетей
__Выполнил:__ *Домченко Максим*

__Студент группы:__ *РИМ-130962*

### Шаг 1: Подготовка данных

In [1]:
# ----------- ШАГ 1.1: Скачивание датасета -----------
# Цель: загрузить данные из Kaggle

# Для локального запуска загрузите kaggle.json в ~/.kaggle/kaggle.json и дайте права:
# !chmod 600 ~/.kaggle/kaggle.json

# Для запуска в colab

# from google.colab import files
# uploaded = files.upload('/root/.kaggle')
# !chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -p ../data -c m5-forecasting-accuracy --force

  0%|                                               | 0.00/45.8M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 45.8M/45.8M [00:00<00:00, 2.93GB/s]


In [2]:
# ----------- ШАГ 1.2: Распаковка архива -----------
# Цель: разархивировать m5-forecasting-accuracy.zip

import zipfile, os

zip_path = "../data/m5-forecasting-accuracy.zip"
extract_dir = "../data"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Архив распакован в:", extract_dir)

Архив распакован в: ../data


In [3]:
# ----------- ШАГ 1.3: Импорты, настройки и пути -----------
# Цель: подключить библиотеки и настроить окружение

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import gc
import warnings

warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid")

# Пути к данным
DATA_DIR = Path("../data")
SAVE_DIR = Path("../data/processed")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# Функция оптимизации памяти
def reduce_mem_usage(df: pd.DataFrame, verbose=True) -> pd.DataFrame:
    start_mem = df.memory_usage(deep=True).sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtype
        if pd.api.types.is_integer_dtype(col_type):
            df[col] = pd.to_numeric(df[col], downcast="integer")
        elif pd.api.types.is_float_dtype(col_type):
            df[col] = pd.to_numeric(df[col], downcast="float")
        elif col_type == object:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage(deep=True).sum() / 1024 ** 2
    if verbose:
        print(f"🔹 Memory usage: {start_mem:.2f} → {end_mem:.2f} MB "
              f"({100 * (start_mem - end_mem) / start_mem:.1f}% savings)")
    return df

In [4]:
# ----------- ШАГ 1.4: Загрузка CSV-файлов -----------
# Цель: считать все таблицы с нужными типами данных

print("Загружаем данные")

dtype_sales = {
    'id': 'category',
    'item_id': 'category',
    'dept_id': 'category',
    'cat_id': 'category',
    'store_id': 'category',
    'state_id': 'category'
}

sales = pd.read_csv(DATA_DIR / "sales_train_validation.csv", dtype=dtype_sales)

calendar = pd.read_csv(
    DATA_DIR / "calendar.csv",
    parse_dates=['date'],
    dtype={
        'weekday': 'category',
        'wm_yr_wk': 'int16',
        'event_name_1': 'category',
        'event_type_1': 'category',
        'event_name_2': 'category',
        'event_type_2': 'category',
        'snap_CA': 'int8',
        'snap_TX': 'int8',
        'snap_WI': 'int8'
    }
)

prices = pd.read_csv(
    DATA_DIR / "sell_prices.csv",
    dtype={
        'store_id': 'category',
        'item_id': 'category',
        'wm_yr_wk': 'int16',
        'sell_price': 'float32'
    }
)


Загружаем данные


In [5]:
# ----------- ШАГ 1.5: Оптимизация памяти -----------
# Цель: минимизировать RAM-потребление

print("Оптимизируем использование памяти:")
sales = reduce_mem_usage(sales)
calendar = reduce_mem_usage(calendar)
prices = reduce_mem_usage(prices)


print("Размеры таблиц:")
for name, df in [('sales', sales), ('calendar', calendar), ('prices', prices)]:
    print(f"  {name:<8}: {df.shape}")


Оптимизируем использование памяти:
🔹 Memory usage: 448.75 → 96.82 MB (78.4% savings)
🔹 Memory usage: 0.19 → 0.21 MB (-15.8% savings)
🔹 Memory usage: 58.96 → 58.96 MB (0.0% savings)
Размеры таблиц:
  sales   : (30490, 1919)
  calendar: (1969, 14)
  prices  : (6841121, 4)


In [6]:
# ----------- ШАГ 1.6: Wide → Long формат + проверка -----------
# Цель: преобразовать таблицу в длинный формат и убедиться, что результат корректен

print("Преобразуем sales в long-формат")

sales_long = pd.melt(
    sales,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

# Размер должен быть 30490 * 1913 = 58327370
expected_rows = sales.shape[0] * (sales.shape[1] - 6)  # 6 id-колонок
print(f"Ожидаемое количество строк: {expected_rows}")
print(f"Полученное количество строк: {sales_long.shape[0]}")
assert sales_long.shape[0] == expected_rows, "Размерность не совпадает!"

print("Пример данных:")
display(sales_long.head())

print("Уникальные значения в 'd':")
print(sales_long['d'].nunique(), "уникальных дат (ожидаем 1913)")
print("Диапазон:", sales_long['d'].min(), "->", sales_long['d'].max())

missing = sales_long.isnull().sum()
print("Пропущенные значения:")
print(missing[missing > 0] if missing.any() else "Нет пропущенных значений")


Преобразуем sales в long-формат
Ожидаемое количество строк: 58327370
Полученное количество строк: 58327370
Пример данных:


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


Уникальные значения в 'd':
1913 уникальных дат (ожидаем 1913)
Диапазон: d_1 -> d_999
Пропущенные значения:
Нет пропущенных значений


In [7]:
# ----------- ШАГ 1.7.1: Добавляем 'wm_yr_wk' в sales_long -----------
# Для последующего join с ценами

print("Добавляем 'wm_yr_wk' в sales_long...")
sales_long = sales_long.merge(
    calendar[['d', 'wm_yr_wk']],
    how='left',
    on='d'
)
print(f"Размер после wm_yr_wk merge: {sales_long.shape}")

# Проверка отсутствия пропусков
assert sales_long['wm_yr_wk'].notnull().all(), "Есть пропуски в 'wm_yr_wk'"
sales_long['wm_yr_wk'].isnull().sum()


Добавляем 'wm_yr_wk' в sales_long...
Размер после wm_yr_wk merge: (58327370, 9)


np.int64(0)

In [8]:
# ----------- ШАГ 1.7.2: Присоединяем цены -----------
# Используем store_id, item_id, wm_yr_wk

print("Присоединяем цены из sell_prices...")
sales_long = sales_long.merge(
    prices,
    how='left',
    on=['store_id', 'item_id', 'wm_yr_wk']
)
print(f"Размер после merge с sell_prices: {sales_long.shape}")

initial_missing = sales_long['sell_price'].isna().sum()
print(f"Пропущено sell_price ДО заполнения: {initial_missing:,}")


Присоединяем цены из sell_prices...
Размер после merge с sell_prices: (58327370, 10)
Пропущено sell_price ДО заполнения: 12,299,413


In [9]:
# ----------- ШАГ 1.7.3: Присоединяем календарь полностью -----------
# Добавляем date, event, snap, weekday, etc.

print("Присоединяем calendar...")
df = sales_long.merge(calendar, how='left', on='d')
print(f"Размер итоговой таблицы после merge с calendar: {df.shape}")

Присоединяем calendar...
Размер итоговой таблицы после merge с calendar: (58327370, 23)


In [10]:
# ----------- ШАГ 1.7.4: Заполнение пропусков в sell_price -----------
# Подтягиваем последнюю известную цену внутри каждой группы (store_id, item_id)

print("Заполняем пропуски в sell_price (по store_id, item_id)...")

df['sell_price'] = (
    df.groupby(['store_id', 'item_id'])['sell_price']
      .apply(lambda s: s.ffill().bfill())
      .reset_index(drop=True)
)

after_fill_missing = df['sell_price'].isna().sum()
print(f"Пропущено sell_price ПОСЛЕ заполнения: {after_fill_missing:,}")
assert after_fill_missing < initial_missing, "Заполнение не сработало"


Заполняем пропуски в sell_price (по store_id, item_id)...
Пропущено sell_price ПОСЛЕ заполнения: 0


In [11]:
# ----------- ШАГ 1.7.5: Финальная очистка и проверка -----------
# Удаляем редкие строки без цены (если вдруг остались)

print("Удаляем строки без цены (если остались)...")
df = df.dropna(subset=['sell_price'])

print(f"Финальный размер df: {df.shape}")
assert df['sell_price'].isna().sum() == 0, "Пропуски остались в sell_price"
print("Проверка пройдена: NaN в sell_price устранены окончательно")

# Пример финальных данных
display(df.head())


Удаляем строки без цены (если остались)...
Финальный размер df: (58327370, 23)
Проверка пройдена: NaN в sell_price устранены окончательно


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk_x,sell_price,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,9.58,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,9.58,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,9.58,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,9.58,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,9.58,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0


### Шаг 2: Генерация признаков (Feature Engineering)

In [12]:
# ----------- ШАГ 2.1: Генерация лагов с сортировкой и float32 -----------

print("Сортируем перед генерацией лагов...")
df = df.sort_values(['id', 'date'])  # <- гарантируем порядок

print("Генерируем лаги: [1, 7, 14, 28, 56]")

LAG_DAYS = [1, 7, 14, 28, 56]

for lag in LAG_DAYS:
    df[f'lag_{lag}'] = (
        df.groupby("id")["sales"]
          .shift(lag)
          .astype("float32")  # ← экономим память
    )

# Проверка
lag_cols = [f'lag_{l}' for l in LAG_DAYS]
missing_counts = df[lag_cols].isna().sum()
total_missing = missing_counts.sum()

print("Пропущенные значения по лагающим признакам:")
print(missing_counts)
print(f"Всего пропущенных в лагах: {total_missing:,}")
display(df[['id', 'date', 'sales'] + lag_cols].head(10))

Сортируем перед генерацией лагов...
Генерируем лаги: [1, 7, 14, 28, 56]
Пропущенные значения по лагающим признакам:
lag_1       30490
lag_7      213430
lag_14     426860
lag_28     853720
lag_56    1707440
dtype: int64
Всего пропущенных в лагах: 3,231,940


,id,date,sales,lag_1,lag_7,lag_14,lag_28,lag_56
0,HOBBIES_1_001_CA_1_validation,2011-01-29,0,NaN,NaN,NaN,NaN,NaN
30490,HOBBIES_1_001_CA_1_validation,2011-01-30,0,0.0,NaN,NaN,NaN,NaN
60980,HOBBIES_1_001_CA_1_validation,2011-01-31,0,0.0,NaN,NaN,NaN,NaN
91470,HOBBIES_1_001_CA_1_validation,2011-02-01,0,0.0,NaN,NaN,NaN,NaN
121960,HOBBIES_1_001_CA_1_validation,2011-02-02,0,0.0,NaN,NaN,NaN,NaN
152450,HOBBIES_1_001_CA_1_validation,2011-02-03,0,0.0,NaN,NaN,NaN,NaN
182940,HOBBIES_1_001_CA_1_validation,2011-02-04,0,0.0,NaN,NaN,NaN,NaN
213430,HOBBIES_1_001_CA_1_validation,2011-02-05,0,0.0,0.0,NaN,NaN,NaN
243920,HOBBIES_1_001_CA_1_validation,2011-02-06,0,0.0,0.0,NaN,NaN,NaN
274410,HOBBIES_1_001_CA_1_validation,2011-02-07,0,0.0,0.0,NaN,NaN,NaN


In [13]:
# ----------- ШАГ 2.2: Базовые скользящие признаки (с shift и min_periods=1) -----------
# Вычисляем rolling-метрики по продажам: mean/std на коротких окнах

print("Генерируем rolling-признаки: rolling_mean_7, rolling_std_14, rolling_mean_28")

ROLLING_WINDOWS = {
    'rolling_mean_7': 7,
    'rolling_std_14': 14,
    'rolling_mean_28': 28,
}

for col_name, window in ROLLING_WINDOWS.items():
    func = np.std if "std" in col_name else np.mean
    df[col_name] = (
        df.groupby("id")["sales"]
          .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).agg(func))
          .astype("float32")
    )

# Проверка
missing_rolling = df[list(ROLLING_WINDOWS)].isna().sum()
total_missing = missing_rolling.sum()

print("Пропущенные значения по rolling-признакам:")
print(missing_rolling)
print(f"Всего пропусков: {total_missing:,}")

# Пример строк
display(df[['id', 'date', 'sales'] + list(ROLLING_WINDOWS)].head(10))

# Заменяем NaN в std-признаке на 0.0
df['rolling_std_14'] = df['rolling_std_14'].fillna(0.0)

# Проверим: все NaN ушли
print(f"Осталось NaN в rolling_std_14: {df['rolling_std_14'].isna().sum()}")


Генерируем rolling-признаки: rolling_mean_7, rolling_std_14, rolling_mean_28
Пропущенные значения по rolling-признакам:
rolling_mean_7     30490
rolling_std_14     60980
rolling_mean_28    30490
dtype: int64
Всего пропусков: 121,960


,id,date,sales,rolling_mean_7,rolling_std_14,rolling_mean_28
0,HOBBIES_1_001_CA_1_validation,2011-01-29,0,NaN,NaN,NaN
30490,HOBBIES_1_001_CA_1_validation,2011-01-30,0,0.0,NaN,0.0
60980,HOBBIES_1_001_CA_1_validation,2011-01-31,0,0.0,0.0,0.0
91470,HOBBIES_1_001_CA_1_validation,2011-02-01,0,0.0,0.0,0.0
121960,HOBBIES_1_001_CA_1_validation,2011-02-02,0,0.0,0.0,0.0
152450,HOBBIES_1_001_CA_1_validation,2011-02-03,0,0.0,0.0,0.0
182940,HOBBIES_1_001_CA_1_validation,2011-02-04,0,0.0,0.0,0.0
213430,HOBBIES_1_001_CA_1_validation,2011-02-05,0,0.0,0.0,0.0
243920,HOBBIES_1_001_CA_1_validation,2011-02-06,0,0.0,0.0,0.0
274410,HOBBIES_1_001_CA_1_validation,2011-02-07,0,0.0,0.0,0.0


Осталось NaN в rolling_std_14: 0


In [14]:
# ----------- ШАГ 2.3: Расширенные rolling-признаки -----------
# Добавляем rolling по большим окнам: 2 месяца, 1 год и минимум за год

print("Генерируем расширенные rolling-признаки: mean_56, mean_365, min_365")

ROLLING_ADVANCED = {
    'rolling_mean_56': ('mean', 56),
    'rolling_mean_365': ('mean', 365),
    'rolling_min_365': ('min', 365),
}

for col_name, (stat, window) in ROLLING_ADVANCED.items():
    func = np.mean if stat == 'mean' else np.min

    df[col_name] = (
        df.groupby("id")["sales"]
          .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).agg(func))
          .astype("float32")
    )

# Проверка наличия пропусков
missing_advanced = df[list(ROLLING_ADVANCED)].isna().sum()
total_missing = missing_advanced.sum()

print("Пропущенные значения по расширенным rolling-признакам:")
print(missing_advanced)
print(f"Всего пропусков: {total_missing:,}")

# Пример строк
display(df[['id', 'date', 'sales'] + list(ROLLING_ADVANCED)].head(10))

# Заполняем пропуски 0.0 — стандарт для rolling min/mean
adv_cols = list(ROLLING_ADVANCED.keys())
df[adv_cols] = df[adv_cols].fillna(0.0)

# Финальная проверка: NaN больше не должно быть
print("NaN в advanced-фичах (после fillna):", df[adv_cols].isna().sum().sum())


Генерируем расширенные rolling-признаки: mean_56, mean_365, min_365
Пропущенные значения по расширенным rolling-признакам:
rolling_mean_56     30490
rolling_mean_365    30490
rolling_min_365     30490
dtype: int64
Всего пропусков: 91,470


,id,date,sales,rolling_mean_56,rolling_mean_365,rolling_min_365
0,HOBBIES_1_001_CA_1_validation,2011-01-29,0,NaN,NaN,NaN
30490,HOBBIES_1_001_CA_1_validation,2011-01-30,0,0.0,0.0,0.0
60980,HOBBIES_1_001_CA_1_validation,2011-01-31,0,0.0,0.0,0.0
91470,HOBBIES_1_001_CA_1_validation,2011-02-01,0,0.0,0.0,0.0
121960,HOBBIES_1_001_CA_1_validation,2011-02-02,0,0.0,0.0,0.0
152450,HOBBIES_1_001_CA_1_validation,2011-02-03,0,0.0,0.0,0.0
182940,HOBBIES_1_001_CA_1_validation,2011-02-04,0,0.0,0.0,0.0
213430,HOBBIES_1_001_CA_1_validation,2011-02-05,0,0.0,0.0,0.0
243920,HOBBIES_1_001_CA_1_validation,2011-02-06,0,0.0,0.0,0.0
274410,HOBBIES_1_001_CA_1_validation,2011-02-07,0,0.0,0.0,0.0


NaN в advanced-фичах (после fillna): 0


In [15]:
# ---------------- Шаг 2.4: ценовые признаки (финальная версия) ----------------
print("Шаг 2.4 — генерируем ценовые признаки без утечки данных")

# 0. Лаг‑1: вчерашняя цена — база для всех расчётов
df['sell_price_lag1'] = (
    df.groupby(['store_id', 'item_id'])['sell_price']
      .shift(1)                      # ← гарантирует прошлое
      .astype('float32')
)

# 1. Годовой максимум цены по прошлым дням
df['price_max_365'] = (
    df.groupby(['store_id', 'item_id'])['sell_price_lag1']
      .rolling(window=365, min_periods=1)
      .max()
      .reset_index(level=[0, 1], drop=True)
      .astype('float32')
)

# 2. Нормированная цена: вчерашняя / прошлый максимум
df['price_norm'] = (df['sell_price_lag1'] / df['price_max_365']).astype('float32')

# 3. Моментум: разница со значением 7 дней назад (исп. лаг‑1)
df['price_momentum_7'] = (
    df['sell_price_lag1'] -
    df.groupby(['store_id', 'item_id'])['sell_price_lag1'].shift(7)
).astype('float32')

# 4. Относительное изменение за 7 дней (в долях)
df['price_pct_change_7'] = (
    df.groupby(['store_id', 'item_id'])['sell_price_lag1']
      .pct_change(7)
).astype('float32')

# 5. Индикатор скидки: цена < 90 % от годового максимума
df['is_discounted'] = (df['price_norm'] < 0.90).astype('int8')

# 6. Уберём вспомогательный столбец
df.drop(columns='sell_price_lag1', inplace=True)

# ── контроль пропусков ────────────────────────────────────────────────────────
price_feats = ['price_max_365', 'price_norm',
               'price_momentum_7', 'price_pct_change_7', 'is_discounted']
print("Пропуски в ценовых признаках:")
print(df[price_feats].isna().sum())

# Результат
display(df[['id', 'date', 'sell_price'] + price_feats].head(10))

Шаг 2.4 — генерируем ценовые признаки без утечки данных
Пропуски в ценовых признаках:
price_max_365          30490
price_norm             30490
price_momentum_7      243920
price_pct_change_7    243920
is_discounted              0
dtype: int64


,id,date,sell_price,price_max_365,price_norm,price_momentum_7,price_pct_change_7,is_discounted
0,HOBBIES_1_001_CA_1_validation,2011-01-29,9.58,NaN,NaN,NaN,NaN,0
30490,HOBBIES_1_001_CA_1_validation,2011-01-30,0.70,9.58,1.000000,NaN,NaN,0
60980,HOBBIES_1_001_CA_1_validation,2011-01-31,0.57,9.58,0.073069,NaN,NaN,1
91470,HOBBIES_1_001_CA_1_validation,2011-02-01,0.48,9.58,0.059499,NaN,NaN,1
121960,HOBBIES_1_001_CA_1_validation,2011-02-02,4.88,9.58,0.050104,NaN,NaN,1
152450,HOBBIES_1_001_CA_1_validation,2011-02-03,3.78,9.58,0.509395,NaN,NaN,1
182940,HOBBIES_1_001_CA_1_validation,2011-02-04,15.48,9.58,0.394572,NaN,NaN,1
213430,HOBBIES_1_001_CA_1_validation,2011-02-05,6.47,15.48,1.000000,NaN,NaN,0
243920,HOBBIES_1_001_CA_1_validation,2011-02-06,4.58,15.48,0.417959,-3.11,-0.324635,1
274410,HOBBIES_1_001_CA_1_validation,2011-02-07,4.88,15.48,0.295866,3.88,5.542857,1


In [16]:
# ----------- Шаг 3.1: Сохраняем подготовленные данные и признаки -----------

# Цель: сохранить очищенные и обогащённые данные + список фичей

print("Сохраняем финальный датасет в Parquet...")

SAVE_PATH = SAVE_DIR / "m5_features_processed.parquet"
df.to_parquet(SAVE_PATH, index=False)  # пока без partition

print(f"Датасет сохранён: {SAVE_PATH}")

Сохраняем финальный датасет в Parquet...
Датасет сохранён: ../data/processed/m5_features_processed.parquet


In [17]:
# ------------ Шаг 3.2: Сохраняем список признаков для моделей ------------
# Цель: сформировать список feature columns (исключая ID, дату и target)

print("Сохраняем список признаков (features.json)...")

feature_cols = sorted([
    col for col in df.columns
    if col not in ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
                   'd', 'date', 'sales', 'wm_yr_wk', 'sell_price']
])

import json

FEAT_PATH = SAVE_DIR / "features.json"
with open(FEAT_PATH, "w") as f:
    json.dump(feature_cols, f, indent=2)

print(f"Список признаков сохранён: {FEAT_PATH}")
print(f"Количество признаков: {len(feature_cols)}")


Сохраняем список признаков (features.json)...
Список признаков сохранён: ../data/processed/features.json
Количество признаков: 29


In [18]:
# ------------ Шаг 3.3: Дополнительно: partition + маски валидации ------------
# Цель: ускорить загрузку (partitioned parquet)

# Сохраняем Parquet с разбиением по store_id (ускорит загрузку по магазинам)
PARTITIONED_PATH = SAVE_DIR / "m5_features_processed_partitioned.parquet"
df.to_parquet(PARTITIONED_PATH, index=False, partition_cols=['store_id'])

print(f"Partitioned Parquet сохранён: {PARTITIONED_PATH}")


Partitioned Parquet сохранён: ../data/processed/m5_features_processed_partitioned.parquet
